In [25]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertModel, BertTokenizer, AutoConfig, BertConfig
from tqdm.auto import tqdm
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score, average_precision_score, accuracy_score, precision_score, recall_score, f1_score, precision_recall_curve
from sklearn.utils.class_weight import compute_class_weight

# A100 specific optimizations
torch.backends.cuda.matmul.allow_tf32 = True  # Enable TF32 for matrix multiplications
torch.backends.cudnn.allow_tf32 = True  

In [26]:
class DTIDataset(Dataset):
    def __init__(self, csv_file, tokenizer, max_length=512):
        self.data = pd.read_csv(csv_file)
        self.tokenizer = tokenizer
        self.max_length = max_length
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        text = row['Target Sequence'] + " [SEP] " + row['SMILES']
        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'token_type_ids': encoding['token_type_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(row['Label'], dtype=torch.long)
        }

In [27]:
class DTIBioBERT(nn.Module):
    def __init__(self, pretrained_model_name='dmis-lab/biobert-v1.1', num_labels=2):
        super().__init__()
        # Paper-specific configuration [1]
        config = BertConfig(
            hidden_size=768,
            num_hidden_layers=6,  # Paper uses 6 transformer layers
            num_attention_heads=12,  # 12 attention heads per layer
            intermediate_size=1536,  # Paper specifies 1536
            hidden_act="gelu",
            hidden_dropout_prob=0.1,
            attention_probs_dropout_prob=0.1,
            max_position_embeddings=512
        )
        
        # Initialize model with paper's architecture [1]
        self.biobert = BertModel(config)
        
        # Classifier matches paper's final layer dimensions [1]
        self.classifier = nn.Sequential(
            nn.Linear(768, 384),  # Hidden size 768 → 384
            nn.GELU(),
            nn.Dropout(0.1),
            nn.Linear(384, num_labels)
        )

    def forward(self, input_ids, token_type_ids, attention_mask):
        with torch.amp.autocast(device_type='cuda', dtype=torch.float16):
            outputs = self.biobert(
                input_ids=input_ids,
                token_type_ids=token_type_ids,
                attention_mask=attention_mask
            )
            pooled_output = outputs.last_hidden_state[:, 0, :]
            logits = self.classifier(pooled_output)
        return logits


In [28]:
def train_model(train_csv, val_csv, model_save_path, epochs=10):
    device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
    
    # Initialize tokenizer and model
    tokenizer = BertTokenizer.from_pretrained('dmis-lab/biobert-v1.1')
    model = DTIBioBERT().to(device)
    
    # Handle class imbalance
    train_labels = pd.read_csv(train_csv)['Label']
    class_weights = compute_class_weight('balanced', classes=np.unique(train_labels), y=train_labels)
    class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)
    criterion = torch.nn.CrossEntropyLoss().to(device).to(torch.float16)  # Match to half precision
    
    # Optimizer and scaler
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5, weight_decay=0.01)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2)
    scaler = torch.cuda.amp.GradScaler()
    
    # Create datasets
    train_dataset = DTIDataset(train_csv, tokenizer)
    val_dataset = DTIDataset(val_csv, tokenizer)
    
    # Create dataloaders with proper settings
    batch_size = 64 if torch.cuda.is_available() else 32
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=4,
        pin_memory=True,
        persistent_workers=True
    )
    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        num_workers=4,
        pin_memory=True,
        persistent_workers=True
    )
    
    best_loss = 1e8
    counter = 0
    
    for epoch in range(epochs):
        # Training
        model.train()
        train_loss = 0
        train_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} [Train]")
        
        for batch in train_bar:
            optimizer.zero_grad()
            
            # Move data to device
            input_ids = batch['input_ids'].to(device, non_blocking=True)
            token_type_ids = batch['token_type_ids'].to(device, non_blocking=True)
            attention_mask = batch['attention_mask'].to(device, non_blocking=True)
            labels = batch['label'].to(device, non_blocking=True)
            
            # Mixed precision training
            with torch.amp.autocast(device_type='cuda', dtype=torch.float16):
                outputs = model(input_ids, token_type_ids, attention_mask)
                loss = criterion(outputs, labels)
            
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            
            train_loss += loss.item()
            train_bar.set_postfix(loss=loss.item())
        
        # Validation
        model.eval()
        val_loss = 0
        all_labels = []
        all_probs = []
        
        val_bar = tqdm(val_loader, desc=f"Epoch {epoch+1}/{epochs} [Val]")
        with torch.no_grad():
            for batch in val_bar:
                input_ids = batch['input_ids'].to(device, non_blocking=True)
                token_type_ids = batch['token_type_ids'].to(device, non_blocking=True)
                attention_mask = batch['attention_mask'].to(device, non_blocking=True)
                labels = batch['label'].to(device, non_blocking=True)
                
                outputs = model(input_ids, token_type_ids, attention_mask)
                loss = criterion(outputs, labels)
                probs = torch.softmax(outputs.float(), dim=1)[:, 1].cpu().numpy()
                
                val_loss += loss.item()
                all_probs.extend(probs)
                all_labels.extend(labels.cpu().numpy())
        
        # Calculate metrics
        precision, recall, thresholds = precision_recall_curve(all_labels, all_probs)
        f1_scores = 2 * (precision * recall) / (precision + recall + 1e-9)
        optimal_idx = np.argmax(f1_scores)
        optimal_threshold = thresholds[optimal_idx] if optimal_idx < len(thresholds) else 0.5
        predictions = (np.array(all_probs) > optimal_threshold).astype(int)
        val_acc = accuracy_score(all_labels, predictions)
        val_auc_roc = roc_auc_score(all_labels, all_probs)
        val_auc_pc = average_precision_score(all_labels, all_probs)
        
        # Calculate confusion matrix for sensitivity and specificity
        tn, fp, fn, tp = confusion_matrix(all_labels, predictions).ravel()
        val_sensitivity = tp / (tp + fn)  # Same as recall
        val_specificity = tn / (tn + fp)
        
        scheduler.step(val_loss)
        
        print(f"\nEpoch {epoch+1}/{epochs}:")
        print(f"Train Loss: {train_loss/len(train_loader):.4f}")
        print(f"Val Loss: {val_loss/len(val_loader):.4f}")
        print(f"Val Accuracy: {val_acc:.4f}")
        print(f"Val AUC ROC: {val_auc_roc:.4f}")
        print(f"Val AUC PC: {val_auc_pc:.4f}")
        print(f"Val Sensitivity: {val_sensitivity:.4f}")
        print(f"Val Specificity: {val_specificity:.4f}")
        
        # Save best model by validation loss
        if val_loss < best_loss:
            best_loss = val_loss
            torch.save(model.state_dict(), model_save_path)
            print("Saved best model by validation loss")
        
        else:
            print("No improvement in validation loss")
            counter += 1
            # if counter >= 15:
            #     print("Early stopping")
            #     return
    
    # Cleanup
    # Proper cleanup
    del batch, input_ids, token_type_ids, attention_mask, labels, outputs, loss
    torch.cuda.empty_cache()

In [29]:
from sklearn.metrics import roc_auc_score, average_precision_score, accuracy_score, confusion_matrix

def evaluate_model(test_csv, model_path):
    device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
    
    # Load model
    tokenizer = BertTokenizer.from_pretrained('dmis-lab/biobert-v1.1')
    model = DTIBioBERT().to(device)
    model.load_state_dict(torch.load(model_path))
    model.eval()
    
    # Create dataset and dataloader
    test_dataset = DTIDataset(test_csv, tokenizer)
    test_loader = DataLoader(
        test_dataset,
        batch_size=64 if torch.cuda.is_available() else 32,
        num_workers=4,
        pin_memory=True
    )
    
    # Evaluation
    all_labels = []
    all_probs = []
    test_bar = tqdm(test_loader, desc="Evaluating")
    
    with torch.no_grad():
        for batch in test_bar:
            input_ids = batch['input_ids'].to(device, non_blocking=True)
            token_type_ids = batch['token_type_ids'].to(device, non_blocking=True)
            attention_mask = batch['attention_mask'].to(device, non_blocking=True)
            labels = batch['label'].to(device, non_blocking=True)
            
            outputs = model(input_ids, token_type_ids, attention_mask)
            probs = torch.softmax(outputs.float(), dim=1)[:, 1].cpu().numpy()
            
            all_probs.extend(probs)
            all_labels.extend(labels.cpu().numpy())
    
    # Calculate metrics using various thresholds
    precision, recall, thresholds = precision_recall_curve(all_labels, all_probs)
    f1_scores = 2 * (precision * recall) / (precision + recall + 1e-9)
    optimal_idx = np.argmax(f1_scores)
    optimal_threshold = thresholds[optimal_idx] if optimal_idx < len(thresholds) else 0.5
    
    # Use the optimal threshold for predictions
    predictions = (np.array(all_probs) > optimal_threshold).astype(int)
    
    # Calculate confusion matrix for sensitivity and specificity
    tn, fp, fn, tp = confusion_matrix(all_labels, predictions).ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    
    metrics = {
        'accuracy': accuracy_score(all_labels, predictions),
        'auc_roc': roc_auc_score(all_labels, all_probs),
        'auc_pc': average_precision_score(all_labels, all_probs),
        'sensitivity': sensitivity,
        'specificity': specificity,
        'optimal_threshold': optimal_threshold
    }
    
    print("\nEvaluation Results:")
    print(f"Test Accuracy: {metrics['accuracy']:.4f}")
    print(f"Test AUC ROC: {metrics['auc_roc']:.4f}")
    print(f"Test AUC PC: {metrics['auc_pc']:.4f}")
    print(f"Test Sensitivity: {metrics['sensitivity']:.4f}")
    print(f"Test Specificity: {metrics['specificity']:.4f}")
    print(f"Optimal Threshold: {metrics['optimal_threshold']:.4f}")
    
    return metrics

In [ ]:
if __name__ == "__main__":
    train_csv = "train.csv"
    val_csv = "val.csv"
    test_csv = "test.csv"
    model_save_path = "biobert_dti_a100.pth"
    
    print("Starting training...")
    train_model(train_csv, val_csv, model_save_path, epochs=50)
    
    print("\nEvaluating on test set...")
    test_metrics = evaluate_model(test_csv, model_save_path)

Starting training...


/tmp/ipykernel_1274980/3351053151.py:17: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()


Epoch 1/50 [Train]:   0%|          | 0/198 [00:00<?, ?it/s]

Epoch 1/50 [Val]:   0%|          | 0/104 [00:00<?, ?it/s]


Epoch 1/50:
Train Loss: 0.6826
Val Loss: 0.6505
Val Accuracy: 0.6702
Val AUC ROC: 0.6570
Val AUC PC: 0.2306
Val Sensitivity: 0.5415
Val Specificity: 0.6911
Saved best model by validation loss


...

Epoch 50/50 [Train]:   0%|          | 0/198 [00:00<?, ?it/s]

Epoch 50/50 [Val]:   0%|          | 0/104 [00:00<?, ?it/s]


Epoch 50/50:
Train Loss: 0.5236
Val Loss: 0.5423
Val Accuracy: 0.8340
Val AUC ROC: 0.7880
Val AUC PC: 0.4422
Val Sensitivity: 0.5102
Val Specificity: 0.8865
No improvement in validation loss

Evaluating on test set...


Evaluating:   0%|          | 0/208 [00:00<?, ?it/s]


Evaluation Results:
Test Accuracy: 0.8170
Test AUC ROC: 0.7795
Test AUC PC: 0.4036
Test Sensitivity: 0.5024
Test Specificity: 0.8696
Optimal Threshold: 0.5505
